[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CLDiego/SPE_GeoHackathon_2025/blob/main/S1_M1_LLM_HF.ipynb)

***
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/write.svg" width="20"/> Follow along by running each cell in order
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> Make sure to run the environment setup cells first
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/reminder.svg" width="20"/> Wait for each installation to complete before proceeding
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/list.svg" width="20" /> Don't worry if installations take a while - this is normal!

In [ ]:
# Environment setup [If running outside Colab]
# !pip install transformers torch matplotlib plotly scikit-learn ipython

# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
# Hugging Face API token
# Retrieving the token is required to get access to HF hub
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

# Session 01 // Module 01: Large Language Models (LLMs) with HuggingFace

In this module, we'll explore the fundamentals of Large Language Models (LLMs) using HuggingFace transformers. We'll cover tokens, embeddings, context windows, and hands-on text generation with a focus on geoscience applications.

## Learning Objectives
- Understand what tokens, embeddings, and context windows are
- Load and use a small HuggingFace model
- Generate simple text completions
- Apply LLMs to geoscience definition tasks

## 1. Understanding Tokens

**Tokens** are the basic units that language models work with. Text is broken down into tokens before being processed by the model. A token can be:
- A whole word (e.g., "seismic")
- Part of a word (e.g., "seis", "mic")
- Punctuation marks
- Special symbols

Let's see how tokenization works with a geoscience example:

In [ ]:
# TODO: Use utils library
from IPython.display import HTML, display
def bert_tokenize_and_color(text, tokenizer ):
    colored_text = ""
    colors = ['#FF5733', '#33FF57', '#3357FF', '#FFD700', '#00CED1', '#FF00FF', '#FFFF00',
              '#FF0000', '#00FF00', '#0000FF', '#00FFFF', '#FF1493', '#8A2BE2',
              '#FF8C00', '#228B22', '#DC143C', '#32CD32', '#1E90FF', '#FFD700', '#FF69B4']

    tokens = tokenizer.tokenize(text)
    colored_html = ""
    
    for i, token in enumerate(tokens):
        color = colors[i % len(colors)]
        # Replace special characters for display
        display_token = token.replace('Ġ', '▁')  # GPT-2 uses Ġ for spaces
        colored_html += f'<span style="background-color:{color}; color: white; padding: 2px 4px; margin: 1px; border-radius: 3px;">{display_token}</span>'
    
    print(f"Original text: {text}")
    print(f"Number of tokens: {len(tokens)}")
    display(HTML(colored_html))
    print(f"Tokens: {tokens}")
    print("-" * 80)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Geoscience text examples
texts = [
    "Seismic inversion is a geophysical technique.",
    "Hydrocarbon exploration uses seismic surveys.",
    "Reservoir characterization involves petrophysical analysis.",
    "What is the porosity and permeability of this formation?"
]

for text in texts:
    bert_tokenize_and_color(text, tokenizer)

In [ ]:
# Display sample vocabulary, special tokens, and token mapping

# Sample vocab (first 20 keys)
vocab = tokenizer.get_vocab()
print("Sample vocabulary (first 20):", list(vocab.keys())[:20])

# Special tokens
print("\nSpecial tokens:", tokenizer.special_tokens_map)

# Mapping for the first line of the poem
sample_text = text[0]
tokens = tokenizer.tokenize(sample_text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"\nSample text: {sample_text}")
print(f"Tokens: {tokens}")
print(f"Token IDs: {token_ids}")

# Full encoding for the first line
encoded = tokenizer(sample_text, return_tensors='pt')
print(f"\nFull encoding (input_ids): {encoded['input_ids']}")
print(f"Attention mask: {encoded['attention_mask']}")

decoded = tokenizer.decode(token_ids)
print(f"Decoded tokens: {decoded}")

Notes:
- You can use `AutoTokenizer` for automatic model selection.
- To perform tokenization, you can use the `tokenizer` object created from the `BertTokenizer` class or the `AutoTokenizer` class.

## 2. Understanding Embeddings

**Embeddings** are numerical representations of tokens in a high-dimensional space. Each token is converted to a vector of numbers that captures its meaning and relationships to other tokens.

Key properties of embeddings:
- Similar words have similar embeddings
- Embeddings capture semantic relationships
- Typical dimensions: 512, 768, 1024, or higher

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import numpy as np

# Load a small model for embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer_embed = AutoTokenizer.from_pretrained(model_name)
model_embed = AutoModel.from_pretrained(model_name)

def get_embeddings(texts, tokenizer, model):
    """Get sentence embeddings"""
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    
    with torch.no_grad():
        outputs = model(**inputs)
        # Use CLS token embedding (first token) for sentence representation
        embeddings = outputs.last_hidden_state[:, 0, :]
    
    return embeddings.numpy()

# Get embeddings for geoscience terms
geoscience_terms = [
    "seismic inversion",
    "reservoir characterization", 
    "hydrocarbon exploration",
    "petrophysical analysis",
    "porosity measurement",
    "permeability analysis"
]

embeddings = get_embeddings(geoscience_terms, tokenizer_embed, model_embed)

print(f"Embedding shape: {embeddings.shape}")
print(f"Each term is represented by {embeddings.shape[1]} numbers")
print(f"\nFirst 10 embedding values for '{geoscience_terms[0]}':")
print(embeddings[0][:10])

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Encode sentences
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    embeddings = model_embed(**inputs).last_hidden_state[:,0,:]  # CLS token

# Reduce dimensions to 3D with t-SNE
tsne = TSNE(n_components=3, perplexity=5, random_state=42)
embeddings_3d = tsne.fit_transform(embeddings.numpy())